In [1]:
# https://youtu.be/tepxdcepTbY
"""
@author: J3viton
Code tested on Tensorflow: 2.2.0
    Keras: 2.4.3
dataset: https://finance.yahoo.com/quote/GE/history/
Also try S&P: https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC
"""


'\n@author: J3viton\nCode tested on Tensorflow: 2.2.0\n    Keras: 2.4.3\ndataset: https://finance.yahoo.com/quote/GE/history/\nAlso try S&P: https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC\n'

In [ ]:


import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
import seaborn as sns

In [ ]:
#Read the csv file
#df = pd.read_csv('GE.csv')
df= pd.read_csv('rovi_05.csv')    ### 'rovi_05.csv''zara_02.csv'
print(df.tail()) #7 columns, including the Date. 


In [ ]:
#Separate dates for future plotting
train_dates = pd.to_datetime(df['Date'])
print(train_dates.tail(5)) #Check last few dates. 

In [ ]:
df.shape

In [ ]:
#Variables for training
cols = list(df)[2:7]  #df.columns.tolist()
#Date and volume columns are not used in training. 
print(cols) #['Open', 'High', 'Low', 'Close', 'Adj Close']

In [ ]:
cols[0]

In [ ]:
#New dataframe with only training data - 5 columns
df_for_training = df[cols].astype(float)

df_for_plot=df_for_training.tail(500)
df_for_plot.plot.line()

In [ ]:
### ESTUDIA como funciona este tema del scaler... deberia ser entre 0 y 1, no?

#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
#Estándariza los datos eliminando la media y escalando los datos de forma que su varianza sea igual a 1.

scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)
df_for_training_scaled


In [ ]:
df_for_training_scaled.shape


In [ ]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainXX = []
trainYY = []
trainX_test=[]

n_future = 6   # origina=1,   Number of days we want to look into the future based on the past days.
n_past = 14  # Number of past days we want to use to predict the future.  FILAS

In [ ]:

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    ## en este caso Append añade un elemento que es un array de dos dimensiones.
    trainXX.append( df_for_training_scaled[ i - n_past : i ,  0:df_for_training.shape[1]])  #n_past filas X 5 columnas (feautures)
    #slicing: fila desde (i-n_past) hasta i///// Columna desde 0: 5 =>(df_for_training.shape[1])
    
    trainYY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 4])  ##[17:18,0] un posicoin de la fila para la columna 0
    ### el 4 es la caracteritica elegida Close//EMA//EMA100//


## separar Training y TEST

In [ ]:
trainX, trainX_test, trainY, trainY_test  = train_test_split(trainXX, trainYY, test_size = 0.30,shuffle = False)


In [ ]:
trainX_test[0:5]

In [ ]:

trainX, trainY = np.array(trainX), np.array(trainY)
trainX_test, trainY_test = np.array(trainX_test), np.array(trainY_test)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))
print('trainX_test shape == {}.'.format(trainX_test.shape))

"""
tenemos un array de 238 elementos en el que cada elemento es un array de 14x5
"""

trainX_test

trainX[0][13][4]

trainY[0][0]

In [ ]:

#In my case, trainX has a shape (12809, 14, 5). 
#12809 because we are looking back 14 days (12823 - 14 = 12809). 
#Remember that we cannot look back 14 days until we get to the 15th day. 
#Also, trainY has a shape (12809, 1). Our model only predicts a single value, but 
#it needs multiple variables (5 in my example) to make this prediction. 
#This is why we can only predict a single day after our training, the day after where our data ends.
#To predict more days in future, we need all the 5 variables which we do not have. 
#We need to predict all variables if we want to do that. 

In [ ]:
# define the Autoencoder model

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()


# fit the model
history = model.fit(trainX, trainY, epochs=20, batch_size=30, validation_split=0.15, verbose=1)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:

#Predicting...
#Libraries that will help us extract only business days in the US.
#Otherwise our dates would be wrong when we look back (or forward).  
#from pandas.tseries.holiday import USFederalHolidayCalendar
#from pandas.tseries.offsets import CustomBusinessDay

#us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
#Remember that we can only predict one day in future as our model needs 5 variables
#as inputs for prediction. We only have all 5 variables until the last day in our dataset.


##n_days_for_prediction=10  #let us predict past 15 days

predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_past, freq='D').tolist()
print(predict_period_dates)

#Make prediction
prediction = model.predict(trainX[0:n_past])
#prediction = model.predict(trainX[-n_days_for_prediction:]) #shape = (n, 1) where n is the n_days_for_prediction
                                                            # desde -n hasta el final. Cada elemento es un array bidimensional
#Pido una predcicion para un array de fechas, me devuelve la predicion para cada una

#Nos vamos n_daysforPredcition atras y calculamos la precidion a n_future (6) days despues.

In [ ]:
#print (trainX[-2:])  # dos ultimos elementos. Cada elemento es un array de 15x5
print (prediction)

In [ ]:
trainY[-n_past+n_future:]

In [ ]:
xx=(range(n_past))

pred= prediction
pred.shape = (n_past)
real=trainY[-n_past:]
real.shape = (n_past)

#sns.lineplot(x=xx, y=pred, color='lightgreen')
#sns.lineplot(x=xx, y=real)

real

In [ ]:
pred

trainX.shape

VISUALIZAION

In [ ]:
pred_gap = np.zeros(n_past+n_future)
muestra_gap =[]
xx=[]

muestreo=15
gapmuestras=np.zeros(muestreo-n_past)
gapprevion=np.zeros(n_future)
gappostprevison= np.zeros(muestreo-n_future-n_past)

for ii in range (len(gapmuestras)):
    gapmuestras[ii] = np.nan
for ii in range (len(gapprevion)):
    gapprevion[ii] = np.nan
for ii in range (len(gappostprevison)):
    gappostprevison[ii] = np.nan    


inicio= np.zeros(n_future+n_past)    

origen=n_past+n_future


for i in range(n_past, len(trainX)-100, muestreo):
    
    prediction = model.predict(trainX[i-n_past:i])
    prediction.shape = (n_past)   #me devuleve 15//n_past previsiones a 6//n_future dias vista
    pred_gap=np.concatenate((pred_gap, prediction), axis=0)  #predcicion son n_past... en un futuro de n_futre muestras
    pred_gap=np.concatenate((pred_gap, gapmuestras), axis=0)

    xx=(trainX[i-n_past:i,0,4])
    muestra_gap=np.concatenate((muestra_gap, xx  ), axis=0)
    muestra_gap=np.concatenate((muestra_gap, gapmuestras), axis=0)

    #print(trainX[i:i+n_days_for_prediction])
    ##print(xx)
    #fake =input()


In [ ]:
%matplotlib notebook

trainX_test.shape


In [ ]:
#sns.lineplot(y=pred_gap, color='lightgreen')

yy2y=trainX[0:(len(trainX)),0,0]
#yy2y.shape = (len(df_for_training_scaled[0]) )
plt.plot(yy2y, label='curva Close',color='lightblue')

yyy=trainX[0:(len(trainX)),0,4]
#yyy.shape = (df_for_training_scaled.shape[0] )
plt.plot(yyy, label='curva REAL',color='pink')

plt.plot(muestra_gap, color='red',label='datos origen Prediccion')
plt.plot(pred_gap, color='lightgreen', label='Predicción')



plt.legend()
plt.show()

In [ ]:
#sns.lineplot(x=range(pred_gap.size), y=pred_gap, color='lightgreen')
#sns.lineplot(x=range(yy.size), y=yy, color='r')

In [ ]:
#plt.savefig('lstm3.jpg',dpi=600)

In [ ]:
yy2y.shape
